# Реализация автоматического дифференцирования

In [2]:
import tensorflow as tf

## 01 example

In [3]:
# создаем переменную
x = tf.Variable(-2.0)

with tf.GradientTape() as tape:
    y = x ** 2

df = tape.gradient(y, x)
print(df)    

tf.Tensor(-4.0, shape=(), dtype=float32)


## 02 exapmle

In [4]:
w = tf.Variable(tf.random.normal((3, 2)))
b = tf.Variable(tf.zeros(2, dtype=tf.float32))
x = tf.Variable([[-2.0, 1.0, 3.0]])

In [5]:
w

<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.05379866, -0.02898621],
       [-0.28529206,  0.9199126 ],
       [-0.5510425 , -0.50802284]], dtype=float32)>

In [6]:
b

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [7]:
x

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float32, numpy=array([[-2.,  1.,  3.]], dtype=float32)>

In [8]:
with tf.GradientTape() as tape:
    y = x @ w + b
    loss = tf.reduce_mean(y ** 2)

df = tape.gradient(loss, [w, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(
[[ 3.6616445  1.092367 ]
 [-1.8308222 -0.5461835]
 [-5.492467  -1.6385505]], shape=(3, 2), dtype=float32)
tf.Tensor([-1.8308222 -0.5461835], shape=(2,), dtype=float32)


В методе `gradient()` первым параметром указываем функцию `loss`, а вторым - список аргументов, от которых вычисляем частные производные в точках `w, b` и при заданном значении

## 03 example

In [9]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.constant(1.5)

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
None


Переменная `b` - это константа, подтому производная по `b` не была вычислена

Преобразуе константу в переменную:

In [10]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)


Запретим отслеживание переменной `x`:

In [11]:
x = tf.Variable(0, dtype=tf.float32, trainable=False)
b = tf.Variable(1.5)

with tf.GradientTape() as tape:
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

None
tf.Tensor(5.0, shape=(), dtype=float32)


Полное отключение отслеживания переменных:

In [12]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

None
None


Укажем переменные, которые следует наблюдать:

In [13]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
None


In [14]:
x = tf.Variable(0, dtype=tf.float32)
b = tf.Variable(1.5)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch([x, b])
    f = (x + b) ** 2 + 2 * b
    
df = tape.gradient(f, [x, b])    
print(df[0], df[1], sep="\n")

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)


## 04 example

Также отслеживание происходит и для всех промежуточных переменных, которые связаны с наблюдаемой переменной.

In [20]:
x = tf.Variable(2, dtype=tf.float32)
y = tf.Variable(3, dtype=tf.float32)

with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x)
    y = 2 * x
    f = y * y
 
df = tape.gradient(f, y)
print(df)

tf.Tensor(8.0, shape=(), dtype=float32)


In [21]:
df = tape.gradient(f, y)

RuntimeError: A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)

Метод `gradient` автоматически высвобождает все ресурсы, связанные с промежуточными вычислениями. Поэтому при повторном его вызове получим ошибку.

Чтобы этого не происходило, при создании объекта `GradientTape()` можно указать параметр `persistent=True`

In [24]:
x = tf.Variable(2, dtype=tf.float32)
y = tf.Variable(3, dtype=tf.float32)

with tf.GradientTape(watch_accessed_variables=False, persistent=True) as tape:
    tape.watch(x)
    y = 2 * x
    f = y * y
 
df = tape.gradient(f, y)
print(df)
df = tape.gradient(f, y)
print(df)

del tape

tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


## 05 example

Если выходная функция является векторной, а ее параметр - скаляр, то результирующий градиент также будет скаларом:

In [25]:
x = tf.Variable(1.0)

with tf.GradientTape() as tape:
    y = [2.0, 3.0] * x ** 2

df = tape.gradient(y, x)
print(df)    

tf.Tensor(10.0, shape=(), dtype=float32)


Произошло суммирование градиентов от каждого выходного значения

А если наоборот, входной параметр является вектором, а функция скаляром, то значения градиентов будут представлены в виде вектора:

In [26]:
x = tf.Variable([1.0, 2.0])

with tf.GradientTape() as tape:
    y = tf.reduce_sum([2.0, 3.0] * x ** 2)

df = tape.gradient(y, x)
print(df)    

tf.Tensor([ 4. 12.], shape=(2,), dtype=float32)


Так как, проходя по графу в обратном направлении, как раз получаем две ведичины для каждого входа.

## 06 example

Функцию можно определять с использованием различных управляющих конструкция языка Python:

In [27]:
x = tf.Variable(1.0)

with tf.GradientTape() as tape:
    if x < 2.0:
        y = tf.reduce_sum([2.0, 3.0] * x ** 2)
    else:
        y = x ** 2

df = tape.gradient(y, x)
print(df)            

tf.Tensor(10.0, shape=(), dtype=float32)


## Особенности вычисления градиентов

Ошибка 1. Неверное определение промежуточной функции

In [29]:
x = tf.Variable(1.0)
y = 2 * x + 1
 
with tf.GradientTape() as tape:
    z = y ** 2
 
df = tape.gradient(z, x)
print(df)

None


Исправим:

In [30]:
x = tf.Variable(1.0)
 
with tf.GradientTape() as tape:
    y = 2 * x + 1
    z = y ** 2
 
df = tape.gradient(z, x)
print(df)

tf.Tensor(12.0, shape=(), dtype=float32)


Ошибка 2. Случайная замена исходной переменной `x` на тензор (константу)

In [33]:
x = tf.Variable(1.0)
 
for n in range(2):
    with tf.GradientTape() as tape:
        y = x ** 2 + 2 * x
 
    df = tape.gradient(y, x)
    print(df)
 
    x = x + 1 # так делать нельзя

tf.Tensor(4.0, shape=(), dtype=float32)
None


После операции `x = x +1` переменная `x` превращается в тензор, которые не отслеживается в `GradientTape()`. Правильнее здесь было бы использовать метод `assing_add()`

In [35]:
x = tf.Variable(1.0)
 
for n in range(2):
    with tf.GradientTape() as tape:
        y = x ** 2 + 2 * x
 
    df = tape.gradient(y, x)
    print(df)
 
    x.assign_add(1.0)

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


Ошибка 3. Использование при вычислении значений не методов Tensorflow, а других спососбов. Например, так:

In [36]:
x = tf.Variable(1.0)
 
with tf.GradientTape() as tape:
    y = tf.constant(2.0) + np.square(x)
 
df = tape.gradient(y, x)
print(df)

NameError: name 'np' is not defined

In [37]:
x = tf.Variable(1.0)
 
with tf.GradientTape() as tape:
    y = tf.constant(2.0) + x * x
 
df = tape.gradient(y, x)
print(df)

tf.Tensor(2.0, shape=(), dtype=float32)


Ошибка 4. Определение переменной с целым типом данных

In [40]:
x = tf.Variable(1)
 
with tf.GradientTape() as tape:
    y = x * x
 
df = tape.gradient(y, x)
print(df)

None


Исправляем:

In [41]:
x = tf.Variable(1.0)
 
with tf.GradientTape() as tape:
    y = x * x
 
df = tape.gradient(y, x)
print(df)

tf.Tensor(2.0, shape=(), dtype=float32)


Ошибка 5. Для целевой функции необходимо прописывать формулы в явном виде

In [42]:
x = tf.Variable(1.0)
w = tf.Variable(2.0)
 
with tf.GradientTape() as tape:
    w.assign_add(x)
    y = w ** 2
 
df = tape.gradient(y, x)
print(df)

None


Исправляем:

In [43]:
x = tf.Variable(1.0)
w = tf.Variable(2.0)
 
with tf.GradientTape() as tape:
    w = w + x
    y = w ** 2
 
df = tape.gradient(y, x)
print(df)

tf.Tensor(6.0, shape=(), dtype=float32)
